In [30]:
import numpy as np

arr = np.array([1 + 2j, 3+1j])

print(arr.max())

(3+1j)


In [29]:
import pandas as pd
import numpy as np

# Sample dataframe (replace with your actual dataframe)
data = {
    'from_account': [101, 101, 101, 101, 101, 101],
    'to_account': [102, 101, 103, 104, 103, 102],
    'amount': [100, 200, 150, 50, 75, 300]
}
df = pd.DataFrame(data)

# Create a pivot table where the rows and columns are account numbers
transaction_matrix = pd.pivot_table(df, 
                                   index='from_account', 
                                   columns='to_account', 
                                   values='amount', 
                                   aggfunc=['mean', 'std'],  # mean # max min std median
                                   fill_value=0)

print(transaction_matrix.xs('mean', level = 0, axis=0))

# Ensure the matrix has all accounts as rows and columns (4x4 matrix)
# Get all unique accounts in sorted order
unique_accounts = sorted(set(df['from_account']).union(set(df['to_account'])))

# Reindex the matrix to match the sorted account list (ensures 4x4 matrix)
transaction_matrix = transaction_matrix.loc['mean'].reindex(index=unique_accounts, columns=unique_accounts, fill_value=0)

# Convert the transaction matrix to a numpy array
transaction_matrix_array = transaction_matrix.to_numpy()
transaction_matrix_array += transaction_matrix_array.T

# Display the resulting 4x4 matrix as a numpy array
print(transaction_matrix_array)

# Calculate the eigenvalues and eigenvectors of A
eigenvalues = np.linalg.eigvals(transaction_matrix_array)
print("Eigenvalues of A:", eigenvalues.max())




TypeError: Index must be a MultiIndex

In [1]:
import numpy as np

# Define the matrix A
A = np.array([[4, 1], 
              [2, 3]])

# Calculate the determinant of A
det_A = np.linalg.det(A)
print("Determinant of A:", det_A)

# Calculate the eigenvalues and eigenvectors of A
eigenvalues, eigenvectors = np.linalg.eig(A)

print("Eigenvalues of A:", eigenvalues)
print("Eigenvectors of A:", eigenvectors)


Determinant of A: 10.000000000000002
Eigenvalues of A: [5. 2.]
Eigenvectors of A: [[ 0.70710678 -0.4472136 ]
 [ 0.70710678  0.89442719]]


In [ ]:
import numpy as np

# Define the matrix A
A = np.array([[0,  2, 1, -4, -5], 
              [2,  0, 0,  0, 0],
              [1,  0, 0,  0, 0],
              [-4, 0, 0,  0, 0],
              [-5, 0, 0,  0, 0]])

# Calculate the determinant of A
det_A = np.linalg.det(A)
print("Determinant of A:", det_A)

# Calculate the eigenvalues and eigenvectors of A
eigenvalues, eigenvectors = np.linalg.eig(A)

print("Eigenvalues of A:", eigenvalues)
print("Eigenvectors of A:")
print(eigenvectors)


Determinant of A: 0.0
Eigenvalues of A: [ 22.04540769 -22.04540769   0.           0.           0.        ]
Eigenvectors of A:
[[ 9.87759340e-02  9.87759340e-02 -1.70445698e-18  3.55574272e-18
  -1.10715782e-19]
 [ 9.94685953e-01 -9.94685953e-01 -4.25177547e-01  8.86981595e-01
   9.24967437e-01]
 [ 4.48056735e-03 -4.48056735e-03  9.05069631e-01  2.78396105e-03
   2.90318686e-03]
 [-1.79222694e-02  1.79222694e-02  5.33800356e-03  4.61586544e-01
  -1.16127474e-02]
 [-2.24028368e-02  2.24028368e-02  6.67250446e-03 -1.39198052e-02
   3.79857810e-01]]


In [32]:
import numpy as np


# Define the matrix A
A = np.array([[1, 5, 2, 1, 4], 
              [5, 2, 0, 0, 9],
              [2, 0, 1, 5, 0],
              [1, 0, 5, 1, 0],
              [4, 9, 0, 0, 1]])


# Calculate the determinant of A
det_A = np.linalg.det(A)
print("Determinant of A:", det_A)

# Calculate the eigenvalues and eigenvectors of A
eigenvalues, eigenvectors = np.linalg.eig(A)

print("Eigenvalues of A:", eigenvalues)
print("Eigenvectors of A:")
print(eigenvectors)


Determinant of A: -6561.000000000005
Eigenvalues of A: [13.84364188  6.30199871 -2.34787471 -7.56204804 -4.23571783]
Eigenvectors of A:
[[ 0.4589336   0.13782702 -0.81529082  0.09039938 -0.3122699 ]
 [ 0.64674377 -0.14805716  0.23633503 -0.70705302  0.06342985]
 [ 0.10062527  0.69126353 -0.10024329 -0.02268766  0.70814252]
 [ 0.07490554  0.67788487  0.39323673  0.00269082 -0.61661893]
 [ 0.59612596 -0.14734187  0.33876656  0.70098645  0.12953543]]


<a href="https://colab.research.google.com/github/Brownwang0426/Reversal-Generative-Reinforcement-Learning/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up (for colab)

In [ ]:
!sudo apt-get install python3.10
!pip install torch==2.0.1 
!pip install numpy==1.25.2 scipy==1.11.4 swig==4.2.1 ufal.pybox2d==2.3.10.3 gymnasium==1.0.0 minigrid==3.0.0 tqdm==4.67.1 dill==0.3.8

In [ ]:
!git clone https://github.com/Brownwang0426/Reversal-Generative-Reinforcement-Learning.git

In [ ]:
import os
os.chdir('/content/Reversal-Generative-Reinforcement-Learning')

# Setting up (for local)
CUDA Toolkit 11.8 \
cuDNN 8.9.x \
pip install torch==2.0.1 --extra-index-url https://download.pytorch.org/whl/cu118  \
pip install numpy==1.25.2 scipy==1.11.4 swig==4.2.1 ufal.pybox2d==2.3.10.3 gymnasium==1.0.0 minigrid==3.0.0 tqdm==4.67.1 dill==0.3.8

# Importing modules

In [1]:
import gymnasium as gym
from gymnasium.wrappers import TimeLimit
import minigrid

import numpy as np
import math
from scipy.special import softmax

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, TensorDataset, Subset

import csv

import multiprocessing as mp
import os
import sys
import copy
import random
import gc
import time
from tqdm import tqdm
from collections import defaultdict

import itertools

import dill

import warnings
warnings.filterwarnings('ignore')

import concurrent.futures
import hashlib

# Checking cuda

In [2]:
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    device_index = 0
    device = torch.device(f"cuda:{device_index}")
    print('using cuda...')
else:
    device = torch.device("cpu")
    print('using cpu...')

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

Device 0: NVIDIA GeForce RTX 4090
using cuda...


# Control board

Crucial configurations regarding how your agent will learn in the environment. The meanings are as follow:
(the configs starting with ⚠️ are what we suggest you must tune according to your specific need in your task)
(the configs starting with ◀️ are what we suggest you to play with to see the effect)



## Configs meaning
| Configs   | Type   | Description                                                                 |
|------------|--------|-----------------------------------------------------------------------------|
| ⚠️game_name  | STR| The name of the environment.                                |
| ⚠️max_steps_for_each_episode | +INT | The maximun steps that the agent will go through while not done. In some environments, it is crucial to increase your "max_steps_for_each_episode" so that your agent can "live long enough" to obatin some better rewards to gradually and heuristically learn better strategy.                    |
| ⚠️seed | +INT/None | The seed for environment. None for random environment each episode.                    |
| load_pretrained_model  | BOLEAN |Whether you want to load previous trained model.                          |
| ◀️ensemble_size  | +INT | The size of the neural ensemble which the agent is comprised of. The bigger, the better, but the longer training time without parallel training. :-D                  |
| ⚠️state_size  | +INT | The size of the state as input data.                    |
| ⚠️action_size   | +INT | The size of action per step as input data.   |
| ⚠️reward_size  | +INT |The size of the reward as output data.                          |
| ⚠️feature_size   | +INT |The size of the hidden layers.       |
| ⚠️history_size  | 0/+INT |How many steps in the history for state and action will the agent take into consideration.                           |
| ⚠️future_size  | +INT |The length of the sequence of actions. Namely, how many steps in the future the agent will predict or use to discern the present best action.                |
| ⚠️neural_type  | STR |  [**`rnn`**, **`gru`**, **`lstm`**, **`td`**] The type of neural network you prefer. For now, we support rnn, gru, lstm, and td (Transformer decoder only). More to come in the future (or you can build one yourself :-D in the models repository).           |
| ⚠️num_layers  | +INT |The number of layers in rnn, gru, lstm, and td (Transformer decoder only).     |
| ⚠️num_heads  | +INT/None |The number of heads in multi-head attention (Should be able to devide feature_size) (Should be None for non-attention neural_type).                         |
| init   | STR | [**`random_normal`**, **`random_uniform`**, **`xavier_normal`**, **`xavier_uniform`**, **`glorot_normal`**, **`glorot_uniform`**] The initialization method you prefer for initiating neural net ensemble of your agent.                          |
| opti   | STR | [**`adam`**, **`sgd`**, **`rmsprop`**]  The optimization method you prefer.             |
| loss  | STR | [**`mean_squared_error`**, **`binary_crossentropy`**] The loss or error function you prefer.                           |
| bias  | BOLEAN |Whether you want add bias.                          |
| drop_rate   | 0/+FLOAT |The drop-rate for drop-out.              |
| ⚠️alpha   | 0/+FLOAT |The learning rate for neural networks weight matrices.                           |
| iteration_for_learning   | +INT |The iteration for learning.              |
| init_   | STR | [**`random_normal`**, **`random_uniform`**, **`xavier_normal`**, **`xavier_uniform`**, **`glorot_normal`**, **`glorot_uniform`**] The initialization method you prefer for initiating actions of your agent.                         |
| greed_epsilon_t  |  +INT |The times applying gaussian noise to the initializated actions of the agent, similar to diffusion model's adding gaussian noise.          |
| ⚠️greed_epsilon_init  |  +FLOAT |The initial greed_epsilon or noise range to initializate the actions of the agent. The higher the value is, the more exploration-oriented the agent will be in the begining.                    |
| ⚠️greed_epsilon_min  |  +FLOAT |A very small number representing the lower bound of the greed_epsilon.        |
| greed_epsilon_decay  |  +FLOAT | The rate of decaying for greed_epsilon for each step and eposide.  |
| ⚠️beta  |  0/+FLOAT |The updating rate for updating actions of the agent.              |
| loss_scale  |  +FLOAT |The scaler for loss in nn for each consecutive time step. The higher, the more weight is given to the loss in later time steps.|
| iteration_for_deducing  |  +INT |The iteration for updating actions of the agent.                           |
| episode_for_training  | +INT |How many epsiodes will your agent run in the training mode where your agent will learn offline.              |
| ⚠️buffer_limit  | +INT |The maximum size for your buffer.              |
| PER_epsilon  | 0/+FLOAT |The epsilon for prioritized experience replay.              |
| PER_exponent  | 0/+FLOAT |The expoenet for prioritized experience replay.                           |
| episode_for_testing  | +INT |How many epsiodes will your agent run in the testing mode where your agent will not learn offline.                        |
| render_for_human  | BOLEAN | Wether you want to render the visual result for each step in the testing mode.              |




## dualism principles

| neural weights | neural actions (and experiences) |
|----------|----------|
| Re-updated   | Not re-updated. New ones are created and stored each time   |
| Stable initialization   | Unstable initialization at first and then stable initialization gradually   |

## frozen lake

In [3]:
game_name =  'FrozenLake-v1'         #⚠️   gym.make(game_name, max_episode_steps=max_steps_for_each_episode, is_slippery=False, map_name="4x4")
max_steps_for_each_episode = 25      #⚠️
seed = None                          #⚠️

load_pretrained_model = True

ensemble_size = 5                    #◀️

state_size =  16                     #⚠️
action_size = 4                      #⚠️
reward_size = 100                    #⚠️
feature_size = 200                   #⚠️
history_size  = 0                    #⚠️
future_size = 10                     #⚠️
neural_type = 'td'                   #⚠️
num_layers = 2                       #⚠️
num_heads = 10                       #⚠️
init = "xavier_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                          #⚠️
iteration_for_learning = 500

init_ = "random_uniform"
greed_epsilon_t     = 1
greed_epsilon_init  = 1.5            #⚠️
greed_epsilon_min   = 0.000001       #⚠️
greed_epsilon_decay = 0.995
beta = 0.1                           #⚠️
loss_scale = 1.1
iteration_for_deducing = 25

episode_for_training = 100000

buffer_limit = 1000                  #⚠️

PER_epsilon  = 0.000001
PER_exponent = 1.1

sequence_size = history_size + future_size 


## blackjack

In [4]:
game_name = 'Blackjack-v1'           #⚠️
max_steps_for_each_episode = 10      #⚠️
seed = None                          #⚠️

load_pretrained_model = True

ensemble_size = 5                    #◀️

state_size =  201                    #⚠️
action_size = 2                      #⚠️
reward_size = 100                    #⚠️
feature_size = 250                   #⚠️
history_size  = 0                    #⚠️
future_size = 5                      #⚠️
neural_type = 'td'                   #⚠️
num_layers = 2                       #⚠️
num_heads = 10                       #⚠️
init = "xavier_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                          #⚠️
iteration_for_learning = 500

init_ = "random_uniform"
greed_epsilon_t     = 1
greed_epsilon_init  = 1.5            #⚠️
greed_epsilon_min   = 0.000001       #⚠️
greed_epsilon_decay = 0.995
beta = 0.1                           #⚠️
loss_scale = 1.1
iteration_for_deducing = 25

episode_for_training = 100000

buffer_limit = 1000                  #⚠️

PER_epsilon  = 0.000001
PER_exponent = 1.1

sequence_size = history_size + future_size 


## cartpole

In [5]:
game_name = 'CartPole-v1'            #⚠️
max_steps_for_each_episode = 1000    #⚠️
seed = None                          #⚠️

load_pretrained_model = True

ensemble_size = 5                    #◀️

state_size =  400                    #⚠️
action_size = 2                      #⚠️
reward_size = 100                    #⚠️
feature_size = 500                   #⚠️
history_size  = 0                    #⚠️
future_size = 25                     #⚠️
neural_type = 'gru'                  #⚠️
num_layers = 2                       #⚠️
num_heads = None                     #⚠️
init = "xavier_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                          #⚠️
iteration_for_learning = 500

init_ = "random_uniform"
greed_epsilon_t     = 1
greed_epsilon_init  = 1.5            #⚠️
greed_epsilon_min   = 0.000001       #⚠️
greed_epsilon_decay = 0.995
beta = 0.1                           #⚠️
loss_scale = 1.1
iteration_for_deducing = 25

episode_for_training = 100000

buffer_limit = 1000                  #⚠️

PER_epsilon  = 0.000001
PER_exponent = 1.1

sequence_size = history_size + future_size 


In [6]:
game_name = 'CartPole-v1'            #⚠️
max_steps_for_each_episode = 1000    #⚠️
seed = None                          #⚠️

load_pretrained_model = True

ensemble_size = 5                    #◀️

state_size =  400                    #⚠️
action_size = 2                      #⚠️
reward_size = 100                    #⚠️
feature_size = 500                   #⚠️
history_size  = 0                    #⚠️
future_size = 25                     #⚠️
neural_type = 'td'                   #⚠️
num_layers = 2                       #⚠️
num_heads = 10                       #⚠️
init = "xavier_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                          #⚠️
iteration_for_learning = 500

init_ = "random_uniform"
greed_epsilon_t     = 1
greed_epsilon_init  = 1.5            #⚠️
greed_epsilon_min   = 0.000001       #⚠️
greed_epsilon_decay = 0.995
beta = 0.1                           #⚠️
loss_scale = 1.1
iteration_for_deducing = 25

episode_for_training = 100000

buffer_limit = 1000                  #⚠️

PER_epsilon  = 0.000001
PER_exponent = 1.1

sequence_size = history_size + future_size 


## mountain car

In [7]:
game_name =  'MountainCar-v0'        #⚠️
max_steps_for_each_episode = 200     #⚠️
seed = None                          #⚠️

load_pretrained_model = True

ensemble_size = 5                    #◀️

state_size =  200                    #⚠️
action_size = 3                      #⚠️
reward_size = 100                    #⚠️
feature_size = 300                   #⚠️
history_size  = 0                    #⚠️
future_size = 25                     #⚠️
neural_type = 'td'                   #⚠️
num_layers = 2                       #⚠️
num_heads = 10                       #⚠️
init = "xavier_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                          #⚠️
iteration_for_learning = 500

init_ = "random_uniform"
greed_epsilon_t     = 1
greed_epsilon_init  = 1.5            #⚠️
greed_epsilon_min   = 0.000001       #⚠️
greed_epsilon_decay = 0.995
beta = 0.1                           #⚠️
loss_scale = 1.1
iteration_for_deducing = 25

episode_for_training = 100000

buffer_limit = 1000                  #⚠️

PER_epsilon  = 0.000001
PER_exponent = 1.1

sequence_size = history_size + future_size 


## acrobot

In [8]:
game_name = 'Acrobot-v1'             #⚠️
max_steps_for_each_episode = 200     #⚠️
seed = None                          #⚠️

load_pretrained_model = True

ensemble_size = 5                    #◀️

state_size =  600                    #⚠️
action_size = 3                      #⚠️
reward_size = 100                    #⚠️
feature_size = 700                   #⚠️
history_size  = 0                    #⚠️
future_size = 25                     #⚠️
neural_type = 'td'                   #⚠️
num_layers = 2                       #⚠️
num_heads = 10                       #⚠️
init = "xavier_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                          #⚠️
iteration_for_learning = 500

init_ = "random_uniform"
greed_epsilon_t     = 1
greed_epsilon_init  = 1.5            #⚠️
greed_epsilon_min   = 0.000001       #⚠️
greed_epsilon_decay = 0.995
beta = 0.1                           #⚠️
loss_scale = 1.1
iteration_for_deducing = 25

episode_for_training = 100000

buffer_limit = 1000                  #⚠️

PER_epsilon  = 0.000001
PER_exponent = 1.1

sequence_size = history_size + future_size 


## lunar lander

In [9]:
game_name = "LunarLander-v3"         #⚠️
max_steps_for_each_episode = 200     #⚠️
seed = None                          #⚠️

load_pretrained_model = True

ensemble_size = 5                    #◀️

state_size =  800                    #⚠️
action_size = 4                      #⚠️
reward_size = 250                    #⚠️
feature_size = 950                   #⚠️
history_size  = 0                    #⚠️
future_size = 25                     #⚠️
neural_type = 'td'                   #⚠️
num_layers = 2                       #⚠️
num_heads = 10                       #⚠️
init = "xavier_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                          #⚠️
iteration_for_learning = 500

init_ = "random_uniform"
greed_epsilon_t     = 1
greed_epsilon_init  = 1.5            #⚠️
greed_epsilon_min   = 0.000001       #⚠️
greed_epsilon_decay = 0.995
beta = 0.1                           #⚠️
loss_scale = 1.1
iteration_for_deducing = 25

episode_for_training = 100000

buffer_limit = 1000                  #⚠️

PER_epsilon  = 0.000001
PER_exponent = 1.1

sequence_size = history_size + future_size 


## door key

In [10]:
game_name = "MiniGrid-DoorKey-5x5-v0"#⚠️
max_steps_for_each_episode = 50      #⚠️
seed = 1                             #⚠️

load_pretrained_model = True

ensemble_size = 5                    #◀️

state_size =  157                    #⚠️
action_size = 7                      #⚠️
reward_size = 100                    #⚠️
feature_size = 250                   #⚠️
history_size  = 15                   #⚠️
future_size = 8                      #⚠️
neural_type = 'td'                   #⚠️
num_layers = 2                       #⚠️
num_heads = 10                       #⚠️
init = "xavier_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                          #⚠️
iteration_for_learning = 500

init_ = "random_uniform"
greed_epsilon_t     = 1
greed_epsilon_init  = 1.5            #⚠️
greed_epsilon_min   = 0.000001       #⚠️
greed_epsilon_decay = 0.995
beta = 0.1                           #⚠️
loss_scale = 1.1
iteration_for_deducing = 25

episode_for_training = 100000

buffer_limit = 1000                  #⚠️

PER_epsilon  = 0.000001
PER_exponent = 1.1

sequence_size = history_size + future_size 


## your present config

In [11]:
game_name = 'CartPole-v1'            #⚠️
max_steps_for_each_episode = 1000    #⚠️
seed = None                          #⚠️

load_pretrained_model = True

ensemble_size = 5                    #◀️

state_size =  400                    #⚠️
action_size = 2                      #⚠️
reward_size = 100                    #⚠️
feature_size = 500                   #⚠️
history_size  = 0                    #⚠️
future_size = 25                     #⚠️
neural_type = 'td'                   #⚠️
num_layers = 2                       #⚠️
num_heads = 10                       #⚠️
init = "xavier_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                          #⚠️
iteration_for_learning = 1000

init_ = "random_uniform"
greed_epsilon_t     = 1
greed_epsilon_init  = 1              #⚠️
greed_epsilon_min   = 0.000001       #⚠️
greed_epsilon_decay = 1
beta = 0.1                           #⚠️
loss_scale = 1.1
iteration_for_deducing = 25

episode_for_training = 100000

buffer_limit = 25000                 #⚠️

PER_epsilon  = 0.000001
PER_exponent = 0.0

sequence_size = history_size + future_size 


In [12]:
episode_for_testing = 100
render_for_human = True

suffix                 = f"game={game_name}_type={neural_type}_ensemble={ensemble_size:05d}_learn={iteration_for_learning:05d}_deduce={iteration_for_deducing:05d}"
directory              = f'./result/{game_name}/'
performance_directory  = f'./result/{game_name}/performace_{suffix}.csv'
model_directory        = f'./result/{game_name}/model_{suffix}.pth'
buffer_directory       = f'./result/{game_name}/buffer_{suffix}.dill'

if not os.path.exists(directory):
    os.makedirs(directory)

# Importing local modules

In [13]:
game_modules = {
    'FrozenLake-v1': 'envs.env_frozenlake',
    'Blackjack-v1': 'envs.env_blackjack',
    'CartPole-v1': 'envs.env_cartpole',
    'MountainCar-v0': 'envs.env_mountaincar',
    'Acrobot-v1': 'envs.env_acrobot',
    'LunarLander-v3': 'envs.env_lunarlander',
    'MiniGrid-DoorKey-5x5-v0': 'envs.env_doorkey'
}
if game_name in game_modules:
    game_module = __import__(game_modules[game_name], fromlist=['vectorizing_state', 'vectorizing_action', 'vectorizing_reward'])
    vectorizing_state  = game_module.vectorizing_state
    vectorizing_action = game_module.vectorizing_action
    vectorizing_reward = game_module.vectorizing_reward
else:
    raise RuntimeError('Missing env functions')

In [14]:
model_modules = {
    'td': 'models.model_td',
    'rnn': 'models.model_rnn',
    'gru': 'models.model_rnn',
    'lstm': 'models.model_rnn'
}
if neural_type in model_modules:
    model_module = __import__(model_modules[neural_type], fromlist=['build_model'])
    build_model  = model_module.build_model
else:
    raise RuntimeError('Missing model functions')

from utils.util_func  import load_performance_from_csv,\
                             load_buffer_from_pickle,\
                             retrieve_history,\
                             retrieve_present,\
                             initialize_future_action, \
                             initialize_desired_reward,\
                             update_future_action, \
                             sequentialize, \
                             update_long_term_experience_replay_buffer,\
                             update_model_list,\
                             limit_buffer,\
                             save_performance_to_csv,\
                             save_buffer_to_pickle


# Deducing -> Learning
Training mode where your agent will learn offline. You can see here how your agent learn overtime and improve its performance.

## Creating or loading models

In [15]:

# creating empty log for recording performance
performance_log  = []

# setting the last episode number for performance log
last_episode = 0

# creating model list
model_list = []
for _ in range(ensemble_size):
    model = build_model(state_size,
                        action_size,
                        reward_size,
                        feature_size,
                        sequence_size,
                        neural_type,
                        num_layers,
                        num_heads,
                        init,
                        opti,
                        loss,
                        bias,
                        drop_rate,
                        alpha)
    model.to(device)
    model_list.append(model)

# creating space for storing tensors as experience replay buffer
history_state_stack        = torch.empty(0).to(device)
history_action_stack       = torch.empty(0).to(device)
present_state_stack        = torch.empty(0).to(device)
future_action_stack        = torch.empty(0).to(device)
future_reward_stack        = torch.empty(0).to(device)
future_state_stack         = torch.empty(0).to(device)
history_state_hash_list    = list()
history_action_hash_list   = list()
present_state_hash_list    = list()
future_action_hash_list    = list()
future_reward_hash_list    = list()
future_state_hash_list     = list()

# load from pre-trained models if needed
if load_pretrained_model == False:
    pass
elif load_pretrained_model == True:
    try:
        model_dict = torch.load(model_directory)
        for i, model in enumerate(model_list):
            model.load_state_dict(model_dict[f'model_{i}'])
        history_state_stack, \
        history_action_stack, \
        present_state_stack, \
        future_action_stack, \
        future_reward_stack, \
        future_state_stack , \
        history_state_hash_list, \
        history_action_hash_list, \
        present_state_hash_list, \
        future_action_hash_list, \
        future_reward_hash_list, \
        future_state_hash_list = load_buffer_from_pickle(buffer_directory)
        history_state_stack    = history_state_stack.to (device) 
        history_action_stack   = history_action_stack.to(device) 
        present_state_stack    = present_state_stack.to (device) 
        future_action_stack    = future_action_stack.to (device) 
        future_reward_stack    = future_reward_stack.to (device) 
        future_state_stack     = future_state_stack .to (device) 
        performance_log        = load_performance_from_csv(performance_directory)
        last_episode           = performance_log[-1][0] + 1 if len(performance_log) > 0 else 0
        greed_epsilon_init     = greed_epsilon_init * (greed_epsilon_decay ** last_episode)
        print('Loaded pre-trained models.')
    except:
        print('Failed loading pre-trained models. Now using new models.')
        pass

Failed loading pre-trained models. Now using new models.


## Putting all the previous works into play

In [ ]:
"""
We dont randomize desired reward anymore because:
1 - It is not typical in RL.
2 - There are many more effective methods like epsilon-greedy, intrinsic motivation, and reward shaping that can drive an agent to explore effectively.
3 - Those methods are designed to balance exploration and exploitation in a way that promotes learning while keeping the agent on a meaningful path toward mastering the environment.
"""

"""
planning phase:
    (history_state) (history_action) present_state         future_action        desired_rewar
                                     -observed by agent    -planned by agent    -given by human
learning phase:
                                     present_state         future_action        future_reward             future_state
                                     -observed by agent    -executed by agent   -observed by agent        -observed by agent
                                                                                -criterion set by human
"""

# starting each episode
for training_episode in tqdm(range(episode_for_training)):

    # initializing summed reward
    summed_reward  = 0

    """
    We filled short term experience replay buffer with some dummy data to insure that history exists.
    """
    # initializing short term experience replay buffer
    state_list  = []
    action_list = []
    reward_list = []
    for _ in range(history_size):
        state_list .append(torch.zeros(state_size  ).to(device) - 1)
        action_list.append(torch.zeros(action_size ).to(device)    )
        reward_list.append(torch.zeros(reward_size ).to(device)    ) 

    # initializing environment
    env            = gym.make(game_name, max_episode_steps=max_steps_for_each_episode)
    state, info    = env.reset(seed = seed)
    
    # observing state
    state          = vectorizing_state(state, device)
    state_list.append(state)

    # starting each step
    for count in itertools.count(1):
        print(f'\rStep: {count}\r', end='', flush=True)

        """
        We let agent took some history states and actions into consideration.
        """
        """
        We also gave loss weight proportional to its position in time step for the agent to be more long-term-reward oriented.
        """
        # initializing and updating action                                    
        history_state, \
        history_action  = retrieve_history(state_list, action_list, history_size, device)
        present_state   = retrieve_present(state_list, device)
        future_action   = initialize_future_action(init_, greed_epsilon_t, greed_epsilon_init, (1, future_size, action_size), device)
        desired_reward  = initialize_desired_reward((1, future_size, reward_size), device)
        future_action   = update_future_action(iteration_for_deducing,
                                               model_list,
                                               history_state ,
                                               history_action,
                                               present_state,
                                               future_action,
                                               desired_reward,
                                               beta,
                                               loss_scale)
        action, action_  = vectorizing_action(future_action, device)
        action_list.append(action)

        # executing action
        state, reward, done, truncated, info = env.step(action_)

        # summing reward
        summed_reward += reward

        # observing actual reward
        reward = vectorizing_reward(state, reward, summed_reward, done, reward_size, device)
        reward_list.append(reward)

        # observing state
        state    = vectorizing_state(state, device)
        state_list.append(state)

        """
        We expanded the condition for terminating an episode to include the case where the count is smaller than the sum of the history and time sizes. 
        Though it is contrary to common practice in RL, this is for better handling the sequentialization of the short-term experience replay buffer with fixed window length.
        """
        # terminating episode if done or truncated
        if count >= (future_size):
            if done or truncated:
                print(f'Episode {training_episode + last_episode}: Summed_Reward = {summed_reward}')
                performance_log.append([training_episode + last_episode, summed_reward])
                break
            else:
                pass
        else:
            pass

    # closing env
    env.close()




    # sequentializing short term experience replay buffer
    history_state_list   ,\
    history_action_list   ,\
    present_state_list   ,\
    future_action_list   ,\
    future_reward_list   ,\
    future_state_list    = sequentialize(state_list  ,
                                         action_list ,
                                         reward_list ,
                                         history_size,
                                         future_size)




    # storing sequentialized short term experience to long term experience replay buffer when it is a new experience
    history_state_stack, \
    history_action_stack, \
    present_state_stack, \
    future_action_stack, \
    future_reward_stack, \
    future_state_stack , \
    history_state_hash_list  , \
    history_action_hash_list  , \
    present_state_hash_list  , \
    future_action_hash_list  , \
    future_reward_hash_list  , \
    future_state_hash_list   = update_long_term_experience_replay_buffer(history_state_stack,
                                                                         history_action_stack,
                                                                         present_state_stack,
                                                                         future_action_stack,
                                                                         future_reward_stack,
                                                                         future_state_stack ,
                                                                         history_state_hash_list  ,
                                                                         history_action_hash_list  ,
                                                                         present_state_hash_list  ,
                                                                         future_action_hash_list  ,
                                                                         future_reward_hash_list  ,
                                                                         future_state_hash_list   ,
                                                                         history_state_list   ,
                                                                         history_action_list   ,
                                                                         present_state_list,
                                                                         future_action_list,
                                                                         future_reward_list,
                                                                         future_state_list )
    



    """
    The true iteration is min(iteration_for_learning, len(present_state_stack))
    """
    # training with Prioritized Experience Replay (PER)
    model_list = update_model_list(iteration_for_learning ,
                                   history_state_stack,
                                   history_action_stack,
                                   present_state_stack,
                                   future_action_stack,
                                   future_reward_stack,
                                   future_state_stack ,
                                   model_list
                                   )




    """
    We also limit the buffer size by using priority to preserve some experiences according to buffer limit.
    """
    # limit_buffer
    history_state_stack, \
    history_action_stack, \
    present_state_stack, \
    future_action_stack, \
    future_reward_stack, \
    future_state_stack , \
    history_state_hash_list  , \
    history_action_hash_list  , \
    present_state_hash_list  , \
    future_action_hash_list  , \
    future_reward_hash_list  , \
    future_state_hash_list   = limit_buffer(history_state_stack,
                                            history_action_stack,
                                            present_state_stack,
                                            future_action_stack,
                                            future_reward_stack,
                                            future_state_stack ,
                                            history_state_hash_list  ,
                                            history_action_hash_list  ,
                                            present_state_hash_list  ,
                                            future_action_hash_list  ,
                                            future_reward_hash_list  ,
                                            future_state_hash_list ,
                                            buffer_limit  )




    # decreasing decay rate
    greed_epsilon_init = greed_epsilon_init * greed_epsilon_decay
    greed_epsilon_init = max( greed_epsilon_min, greed_epsilon_init )




    # saving final reward to log
    save_performance_to_csv(performance_log, performance_directory)

    # saving nn models
    model_dict = {}
    for i, model in enumerate(model_list):
        model_dict[f'model_{i}'] = model.state_dict()
    torch.save(model_dict, model_directory)

    # saving long term experience replay buffer
    save_buffer_to_pickle(buffer_directory,
                            history_state_stack,
                            history_action_stack,
                            present_state_stack,
                            future_action_stack,
                            future_reward_stack,
                            future_state_stack,
                            history_state_hash_list,
                            history_action_hash_list,
                            present_state_hash_list,
                            future_action_hash_list,
                            future_reward_hash_list,
                            future_state_hash_list)




    # clear up
    gc.collect()
    torch.cuda.empty_cache()

  0%|          | 0/100000 [00:00<?, ?it/s]

Episode 0: Summed_Reward = 10.0


  0%|          | 1/100000 [00:58<1630:48:13, 58.71s/it]

Episode 1: Summed_Reward = 39.0


  0%|          | 2/100000 [02:47<2445:33:25, 88.04s/it]

...

  0%|          | 221/100000 [47:09:31<38688:14:26, 1395.86s/it]

Episode 221: Summed_Reward = 285.0


  0%|          | 222/100000 [47:26:41<35651:30:32, 1286.31s/it]

Episode 222: Summed_Reward = 180.0


  0%|          | 223/100000 [47:40:52<32032:58:16, 1155.76s/it]

# Deducing only
Testing mode where your trained agent in the training mode will not learn offline. It just keeps running each episode without learning new stuff.

## Loading models

In [ ]:
model_list = []
for _ in range(ensemble_size):
    model = build_model(state_size,
                        action_size,
                        reward_size,
                        feature_size,
                        sequence_size ,
                        neural_type,
                        num_layers,
                        num_heads,
                        init,
                        opti,
                        loss,
                        bias,
                        drop_rate,
                        alpha)
    model.to(device)
    model_list.append(model)

model_dict = torch.load(model_directory)
for i, model in enumerate(model_list):
    model.load_state_dict(model_dict[f'model_{i}'])

## Putting all the previous works into play ... again

But this time the agent does not learn

In [ ]:
# score recorder
total_summed_reward = 0

# starting each episode
for testing_episode in range(episode_for_testing):

    # stabilizing decay rate
    greed_epsilon = copy.deepcopy(greed_epsilon_min)

    # initializing summed reward
    summed_reward  = 0

    # initializing short term experience replay buffer
    state_list  = []
    action_list = []
    for _ in range(history_size):
        state_list .append(torch.zeros(state_size  ).to(device) - 1)
        action_list.append(torch.zeros(action_size ).to(device)    )

    # initializing environment
    env = gym.make(game_name, max_episode_steps = max_steps_for_each_episode,
                   render_mode = "human" if render_for_human else None)
    state, info = env.reset(seed = seed)
    if render_for_human == True:
        env.render()

    # observing state
    state = vectorizing_state(state, device)
    state_list.append(state)

    # starting each step
    for count in itertools.count(1):
        print(f'\rStep: {count}\r', end='', flush=True)
        
        # initializing and updating action   
        history_state, \
        history_action = retrieve_history(state_list, action_list, history_size, device)
        present_state  = retrieve_present(state_list, device)
        future_action  = initialize_future_action(init_, greed_epsilon_t, greed_epsilon, (1, future_size, action_size), device)
        desired_reward = initialize_desired_reward((1, future_size, reward_size), device)
        future_action  = update_future_action(iteration_for_deducing,
                                              model_list,
                                              history_state ,
                                              history_action,
                                              present_state,
                                              future_action,
                                              desired_reward,
                                              beta,
                                              loss_scale)
        action, action_       = vectorizing_action(future_action, device)
        action_list.append(action)
        
        # executing action
        state, reward, done, truncated, info = env.step(action_)
        if render_for_human == True:
            env.render()
            
        # summing reward
        summed_reward += reward
        
        # observing state
        state = vectorizing_state(state, device)
        state_list.append(state)
        
        # terminating episode if done or truncated
        if done or truncated:
            break
        
    # closing env
    env.close()

    # recording
    print("Summed reward:", summed_reward)
    print(f'Episode: {testing_episode + 1}')
    print('Everaged summed reward:')
    total_summed_reward += summed_reward
    print(total_summed_reward/(testing_episode + 1))

